In [3]:
import boto3
import time
import json
from urllib.parse import urlparse

In [6]:
aws_region = 'us-east-1'

def start_transcription_job(job_name, media_uri, output_bucket_name, region_name=aws_region):
    transcribe_client = boto3.client('transcribe', region_name=aws_region)
    response = transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': media_uri}, 
        MediaFormat='mp3',  
        LanguageCode='hi-IN',   
        OutputBucketName=output_bucket_name
        )
    return response

def get_transcription_job_status(job_name, region_name=aws_region):
    transcribe_client = boto3.client('transcribe', region_name=region_name)
    response = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
    return response['TranscriptionJob']['TranscriptionJobStatus']

def get_transcription_result(job_name, region_name=aws_region):
    transcribe_client = boto3.client('transcribe', region_name=aws_region)
    response = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
    transcript_uri = response['TranscriptionJob']['Transcript']['TranscriptFileUri']
    parsed_url = urlparse(transcript_uri)
    bucket_name = parsed_url.netloc.split('.')[0]  
    key = parsed_url.path.lstrip('/')  
    return f"s3://{key}",bucket_name
    
def transcribe_audio(media_uri, output_bucket_name):
    job_name = f"transcription-job-{int(time.time())}"
    start_response = start_transcription_job(job_name, media_uri, output_bucket_name)
    print(f"Started transcription job: {job_name}")
    
    while True:
        status = get_transcription_job_status(job_name)
        if status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} finished with status: {status}")
            break
        print(f"Job {job_name} is still in progress...")
        time.sleep(10)

    if status == 'COMPLETED':
        transcript_uri,bucket_name = get_transcription_result(job_name)
        print(f"Transcription result available at: {transcript_uri}","bucket_name: ",bucket_name)
        return transcript_uri,bucket_name
    else:
        return (f"Transcription job {job_name} failed."),None

In [7]:
media_uri = 's3://email-transcription-input/hindi2.mp3'  # Replace with your actual S3 URI
output_bucket_name = 'email-transcription-ip-translation-op'  # Replace with your actual S3 bucket name
s3_uri,bucket=transcribe_audio(media_uri, output_bucket_name)

Started transcription job: transcription-job-1725266564
Job transcription-job-1725266564 is still in progress...
Job transcription-job-1725266564 finished with status: COMPLETED
Transcription result available at: s3://email-transcription-ip-translation-op/transcription-job-1725266564.json bucket_name:  s3


In [8]:
# Initialize an S3 client
s3 = boto3.client('s3')
# Initialize Bedrock client
aws_region = 'us-east-1'
bedrock = boto3.client(service_name='bedrock-runtime',region_name=aws_region)

def transcribe_json(s3_uri):
    
    s3 = boto3.client('s3')
    bucket_name = "email-transcription-ip-translation-op"
    key = s3_uri.split("/")[-1]
    
    # Get the object from S3
    response = s3.get_object(Bucket=bucket_name, Key=key)
    json_data = response['Body'].read().decode('utf-8')

    # Load the string as JSON
    data = json.loads(json_data)

    # Return the transcript
    
    return data["results"]["transcripts"][0]["transcript"],key
s3_uri = "s3://email-transcription-ip-translation-op/transcription-job-1725266564.json"
transcript,key=transcribe_json(s3_uri)
print(transcript)
# function for the translation




नमस्कार हमारे area में जो road की condition है वो बहुत खराब हो चुकी है । उसके बारे में municipality को एक email लिखना है, जिसमें road के संबंधित तकरार करनी है और road की condition बहुत जल्द अच्छी हो जाए इसके लिए एक mail लिखना है । mail descriptive होना चाहिए, subjective भी होना चाहिए । Thank you ।


In [9]:
def translate(s3_uri):
    text,key=transcribe_json(s3_uri)
    if len(text.split(" "))<1000:
        tokens=1000
    else:
        tokens=int((len(text.split(" ")))*1.5)
# Prepare the request body
    body = json.dumps({
        "max_tokens": tokens,
        "temperature": 0,
        "system": """You are a good language translator.
        1. Translate the content into english language. 
        2. The content is totally based on the email content, 
        3. Understand the user prompt/requirement and thereby give the requested output professionally.
        4. Make it look more compact and modified Email content and detailed if necessary.
        5. Correct if there is any mistake""",
        "messages": [

            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": text
                    }
                ]
            },
            {
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        "text": "Sure, I will follow the instructions. I am translating content to English and making it more compact and modified."
                    }
                ]
            },
        ],
        "anthropic_version": "bedrock-2023-05-31"
    })

    # Call Bedrock API with the appropriate model ID
    response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0")

    # Parse and print the response
    response_body = json.loads(response.get("body").read())
    return response_body["content"][0]["text"],key

s3_uri="s3://email-transcription-ip-translation-op/transcription-job-1725266564.json"
output_string,key=translate(s3_uri)
print(output_string)



Subject: Urgent Need for Road Repair in Our Area

Dear Sir/Madam,

I hope this email finds you well. I am writing to bring to your attention the deplorable condition of the roads in our area, which has become a significant concern for the residents.

The roads in our neighborhood have deteriorated to an alarming state, with numerous potholes, cracks, and uneven surfaces. These hazardous conditions pose a severe risk to pedestrians, cyclists, and motorists alike, making it challenging and unsafe to navigate through the area.

Furthermore, the poor road conditions have led to increased wear and tear on vehicles, resulting in additional expenses for residents. The situation becomes even more problematic during inclement weather, as the damaged roads are prone to water accumulation, creating potential hazards and further exacerbating the existing issues.

I kindly request that the municipality take immediate action to address this pressing matter. A comprehensive road repair and maintena

In [36]:
def save_to_s3(output_string,key):
    # output bucket for text file
    output_bucket_name = "email-translation-output"
    output_file_key = key.split(".")[0]+".txt"

    output_string = output_string

    s3.put_object(Bucket=output_bucket_name, Key=output_file_key, Body=output_string)

    return f"File saved to S3 bucket {bucket_name} with key {output_file_key}"